In [ ]:
import folium
import random
import time
from datetime import datetime
from IPython.display import display, HTML, clear_output
import pandas as pd

# List of major cities around the world with simulated weather patterns
CITIES = [
    {"name": "Hanoi", "lat": 21.0285, "lon": 105.8542, "weather_patterns": ["sunny", "light rain", "cloudy", "partly sunny"]},
    {"name": "Tokyo", "lat": 35.6762, "lon": 139.6503, "weather_patterns": ["sunny", "rainy", "cloudy", "foggy"]},
    {"name": "New York", "lat": 40.7128, "lon": -74.0060, "weather_patterns": ["sunny", "cloudy", "snowy", "rainy"]},
    {"name": "London", "lat": 51.5074, "lon": -0.1278, "weather_patterns": ["rainy", "foggy", "cloudy", "partly sunny"]},
    {"name": "Paris", "lat": 48.8566, "lon": 2.3522, "weather_patterns": ["sunny", "light rain", "cloudy", "foggy"]},
    {"name": "Sydney", "lat": -33.8688, "lon": 151.2093, "weather_patterns": ["sunny", "rainy", "windy", "cloudy"]},
    {"name": "Cairo", "lat": 30.0444, "lon": 31.2357, "weather_patterns": ["sunny", "windy", "hot", "dusty"]},
    {"name": "Rio de Janeiro", "lat": -22.9068, "lon": -43.1729, "weather_patterns": ["sunny", "showers", "partly sunny", "cloudy"]}
]

def get_simulated_weather(city):
    """Generate simulated weather data"""
    # Base temperature on location for more realistic values
    lat = abs(city["lat"])
    
    # Lower temperatures at the poles, higher at the equator
    base_temp = 30 - abs(lat - 20) * 0.6
    
    # Add random fluctuation
    temp = base_temp + random.uniform(-5, 5)
    
    # Round temperature
    temp = round(temp, 1)
    
    # Randomly select weather description from the city's patterns
    weather_desc = random.choice(city["weather_patterns"])
    
    # Generate other weather parameters
    wind_speed = round(random.uniform(0, 10), 1)
    humidity = random.randint(30, 90)
    
    return {
        "temp": temp,
        "weather_desc": weather_desc,
        "wind_speed": wind_speed,
        "humidity": humidity
    }

def create_weather_map():
    """Create a weather map and display it in Colab"""
    # Create world map
    weather_map = folium.Map(location=[20, 0], zoom_start=2, tiles="OpenStreetMap")
    
    # Weather information table for display in Colab
    weather_data_list = []
    
    # Add weather information for each city
    for city in CITIES:
        weather_data = get_simulated_weather(city)
        
        temp = weather_data["temp"]
        weather_desc = weather_data["weather_desc"]
        wind_speed = weather_data["wind_speed"]
        humidity = weather_data["humidity"]
        
        # Add to weather information list
        weather_data_list.append({
            "City": city["name"],
            "Temperature (°C)": temp,
            "Weather": weather_desc,
            "Wind Speed (m/s)": wind_speed,
            "Humidity (%)": humidity
        })
        
        # Choose icon color based on temperature
        if temp < 0:
            color = "blue"
        elif temp < 15:
            color = "green"
        elif temp < 30:
            color = "orange"
        else:
            color = "red"
        
        # Create weather information popup
        popup_html = f"""
        <div style="width: 200px">
            <h4>{city['name']}</h4>
            <p><b>Temperature:</b> {temp}°C</p>
            <p><b>Weather:</b> {weather_desc}</p>
            <p><b>Wind Speed:</b> {wind_speed} m/s</p>
            <p><b>Humidity:</b> {humidity}%</p>
            <p><i>Updated: {datetime.now().strftime('%H:%M:%S')}</i></p>
        </div>
        """
        
        # Add marker to the map
        folium.Marker(
            location=[city["lat"], city["lon"]],
            popup=folium.Popup(popup_html, max_width=300),
            tooltip=f"{city['name']}: {temp}°C, {weather_desc}",
            icon=folium.Icon(color=color)
        ).add_to(weather_map)
    
    # Add title and information to the map
    title_html = '''
    <div style="position: fixed; 
                top: 10px; left: 50px; width: 300px; height: 60px; 
                background-color: white; border:2px solid grey; z-index:9999; 
                font-size:16px; padding: 8px;">
        <b>World Weather Map</b><br>
        Updated at: {0}
    </div>
    '''.format(datetime.now().strftime("%m/%d/%Y %H:%M:%S"))
    
    weather_map.get_root().html.add_child(folium.Element(title_html))
    
    # Create DataFrame from weather data
    weather_df = pd.DataFrame(weather_data_list)
    
    return weather_map, weather_df

def update_weather_in_colab(num_updates=5, update_interval=10):
    """Update weather in Colab for a specific number of times at set intervals"""
    for i in range(num_updates):
        # Clear previous output
        clear_output(wait=True)
        
        # Create new map and data table
        weather_map, weather_df = create_weather_map()
        
        # Display update time
        current_time = datetime.now().strftime("%m/%d/%Y %H:%M:%S")
        print(f"Weather update {i+1}/{num_updates} - {current_time}")
        
        # Display map
        display(weather_map)
        
        # Display data table
        display(HTML("<h3>Weather Information Table</h3>"))
        display(weather_df.style.background_gradient(cmap='viridis', subset=['Temperature (°C)']))
        
        # Wait until next update (except for the last one)
        if i < num_updates - 1:
            print(f"Waiting {update_interval} seconds until next update...")
            time.sleep(update_interval)

# Run the update function with 5 updates, 10 seconds apart
update_weather_in_colab(num_updates=5, update_interval=10)